# Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import cosmology
from astropy.cosmology import FlatLambdaCDM
from Corrfunc.theory import DDsmu
from scipy.special import eval_legendre

In [2]:
def ReadFits(fn):
    with fits.open(fn) as hdul:
        # Assuming the data is in the first extension (index 1)
        data = hdul[1].data
    return data

def sky_to_cartesian(data, cosmo):
    ra = data['RA']
    dec = data['DEC']
    z = data['Z']
    
    # Convert to Cartesian coordinates
    ra_rad = np.deg2rad(ra)
    dec_rad = np.deg2rad(dec)
    comoving_distance = cosmo.comoving_distance(z).value  # in Mpc
    
    x = comoving_distance * np.cos(dec_rad) * np.cos(ra_rad)
    y = comoving_distance * np.cos(dec_rad) * np.sin(ra_rad)
    z = comoving_distance * np.sin(dec_rad)
    
    return [x,y,z]

# Getting eBOSS survey window function

In [ ]:
file_eBOSS = np.load('CARTESIAN_0.8_2.2_eBOSS_QSO_clustering_random-NGC-vDR16.npy')
eBOSS_weight, eBOSS_X, eBOSS_Y, eBOSS_Z =  file_eBOSS.T

In [ ]:
N_eBOSS = eBOSS_weight.shape[0]
frac = 0.1
mask_eBOSS = decrease_cat(frac, N_eBOSS)

In [ ]:
#Number of mu bins
Nmu_bins = 1000

#specs for the separation array
smin = 1
smax = 100
sbins = 100

#mu bins
mu_edges = np.linspace(0,1,Nmu_bins + 1)
mu_ctrs = 0.5*(mu_edges[1:] + mu_edges[:-1])

#separation edges
s_edges = np.logspace(np.log10(smin),np.log10(smax),sbins)
s_ctrs = 0.5*(s_edges[1:]+ s_edges[:-1]) 

In [ ]:
result_eBOSS = DDsmu(1,1, s_edges, 1, Nmu_bins, eBOSS_X[mask_eBOSS],eBOSS_Y[mask_eBOSS],eBOSS_Z[mask_eBOSS],
                     weights1 = eBOSS_weight[mask_eBOSS],weight_type='pair_product',  periodic = False)

In [ ]:
Q_ell_eBOSS = Q_ell_estimate(result_eBOSS, s_ctrs, mu_ctrs)

In [ ]:
plt.figure()
plt.semilogx(s_ctrs, Q_ell_eBOSS[0]/Q_ell_eBOSS[0][20])

# Getting the DESI survey window function

In [ ]:
randoms = ReadFits('desi_survey_catalogs_Y1_mocks_SecondGenMocks_AbacusSummit_v4_2_mock0_QSO_complete_NGC_0_clustering.ran.fits')

In [ ]:
# Specify the cosmology. Using the DESI cosmology
cosmo = FlatLambdaCDM(H0=67.36, Om0=0.3137721026737606)
zmin = 0.8
zmax = 2.1

mask = (randoms['Z']>=zmin)&(randoms['Z']<=zmax)
randoms = randoms[mask]

In [ ]:
X,Y,Z = sky_to_cartesian(randoms, cosmo)
WEIGHT = randoms['WEIGHT']

In [ ]:
output = np.column_stack((X,Y,Z,WEIGHT))

In [ ]:
np.save('cartesian_desi_survey_catalogs_Y1_mocks_SecondGenMocks_AbacusSummit_v4_2_mock0_QSO_complete_NGC_0_ran.npy', output)

# eBOSS

In [3]:
data = np.load('eBOSS/QSO/cat/CARTESIAN_0.8_2.2_eBOSS_QSO_clustering_random-NGC-vDR16.npy')

In [4]:
W,X,Y,Z = data.T

In [7]:
np.save('eBOSS/QSO/cat/CARTESIAN_0.8_2.2_eBOSS_QSO_clustering_random-NGC-vDR16_new.npy', np.column_stack((X,Y,Z,W)))